In [264]:
#Import statements
import json
import requests
import pandas as pd
import numpy as np
import datetime
from pandas.io.json import json_normalize
import os
import csv
import time
import pickle

In [265]:
stockSymbol = ["AAPL", "AMZN", "GOOGL","MSFT", "DELL", "IBM", "INTC", "HPQ",
               "FB", "CSCO", "ORCL", "HPE", "MU", "DXC", "TMO"]

In [266]:
fullDf = pd.DataFrame(columns=['id', 'body', 'tag'])
for symbol in stockSymbol:
    df = pd.read_csv("{}_twitsCopy.csv".format(symbol), sep=",", usecols=(0,1,3))
    fullDf = fullDf.append(df)
fullDf = fullDf.drop_duplicates('id')
fullDf = fullDf.reset_index()[{'id','body','tag'}]
#fullDf = fullDf[{'id','body','tag'}]

In [267]:
len(fullDf['id'].unique())

101769

In [268]:
bullish = fullDf[fullDf['tag'] == 'Bullish'].reset_index()[{'id','body','tag'}]
bearish = fullDf[fullDf['tag'] == 'Bearish'].reset_index()[{'id','body','tag'}]
none = fullDf[fullDf['tag'] == 'none'].reset_index()[{'id','body','tag'}]
taggedDf = bullish.append(bearish)

In [269]:
print(len(bullish))
print(len(bearish))

26230
6909


In [270]:
len(bullish)/len(taggedDf)

0.7915145297081988

In [228]:
#balance data
np.random.seed(42)
rand = np.random.permutation(bullish.shape[0])
bullish = bullish.iloc[rand[:bearish.shape[0]]].reset_index(drop=True)

In [229]:
taggedDf = bullish.append(bearish)

In [230]:
bullish.to_csv(r'bullish_twits.csv', index = False)
bearish.to_csv(r'bearish_twits.csv', index = False)
none.to_csv(r'none_twits.csv', index = False)

In [271]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(taggedDf['body'].values, taggedDf['tag'].values, test_size=0.2, random_state=42)

In [272]:
#### This cell or the following. You choose
import re
import string
# Create a token pattern.  It is better to use re.compile, 
# as it will apply to multiple matches, which makes it faster
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): 
    return re_tok.sub(r' \1 ', s).split()

In [273]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=tokenize)

In [274]:
vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x00000295B420A048>,
                use_idf=True, vocabulary=None)

In [275]:
tf_train = vect.fit_transform(X_train)
tf_test = vect.transform(X_test)

In [276]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(tf_train, y_train)

In [277]:
# get probabilities for positive class
probs = model.predict_proba(tf_test)[:,1]

In [278]:
# get predictions
predictions = model.predict(tf_test)

In [279]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score 
from sklearn.metrics import confusion_matrix,roc_curve, auc

In [280]:
# Report the predctive performance metrics
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print(accuracy)

0.7948098974049487


In [281]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=0, solver='liblinear',multi_class='ovr').fit(tf_train, y_train)

In [282]:
# get probabilities for positive class
probs = model.predict_proba(tf_test)[:,1]
probs

array([0.85443635, 0.86826171, 0.96103891, ..., 0.81120568, 0.88722831,
       0.88378021])

In [283]:
predictions = model.predict(tf_test)
predictions

array(['Bullish', 'Bullish', 'Bullish', ..., 'Bullish', 'Bullish',
       'Bullish'], dtype=object)

In [284]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score 
from sklearn.metrics import confusion_matrix,roc_curve, auc

In [285]:
# Report the predctive performance metrics
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
accuracy

0.8427881713940857

In [286]:
accuracy

0.8427881713940857

In [287]:
cm = confusion_matrix(y_test, predictions)
print(f"Confusion matrix.:\n {cm}")

Confusion matrix.:
 [[ 478  907]
 [ 135 5108]]


In [248]:
xNew = none

In [288]:
tf_new = vect.transform(none['body'])

In [289]:
probs = model.predict_proba(tf_new)[:,1]
probs

array([0.92202899, 0.84939803, 0.69730939, ..., 0.86172861, 0.99262798,
       0.9053213 ])

In [290]:
predictions = model.predict(tf_new)
predictions

array(['Bullish', 'Bullish', 'Bullish', ..., 'Bullish', 'Bullish',
       'Bullish'], dtype=object)

In [291]:
len(predictions[predictions == 'Bullish']), len(predictions[predictions == 'Bearish'])

(65020, 3610)

In [292]:
len(predictions[predictions == 'Bullish'])/(len(predictions[predictions == 'Bullish'])+ len(predictions[predictions == 'Bearish']))

0.9473990966049832